# read packages and dataset

In [1]:
from embeddings import * # read the codes and the comments in the embeddings package please, some small tweaks may have to be made, depending on the task at hand. 
import pandas as pd
from tqdm import tqdm
import pickle
# this line prevents warnings from popping up 
# import warnings
# warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import RepeatedKFold, cross_val_predict, cross_val_score
import numpy as np
from numpy import mean, std
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, make_scorer, average_precision_score
from tqdm import tqdm
import statistics
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import os
from sklearn.model_selection import StratifiedKFold

if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

/home/charles/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-11-01 03:22:57.052871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 03:22:57.623179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


# read and inspect dataset

In [3]:
df=pd.read_csv("df.csv")
len(df)

84875

In [4]:
(df['DVT']).value_counts()

False    84377
True       498
Name: DVT, dtype: int64

In [11]:
(df['PE']).value_counts()

False    84588
True       287
Name: PE, dtype: int64

In [12]:
(df['death_in_30']).value_counts()

False    83181
True      1694
Name: death_in_30, dtype: int64

In [13]:
(df['PNA']).value_counts()

False    84400
True       475
Name: PNA, dtype: int64

In [14]:
(df['post_aki_status']).value_counts()

0    73457
1    11418
Name: post_aki_status, dtype: int64

In [15]:
(df['postop_del']).value_counts()

False    6483
True     5695
Name: postop_del, dtype: int64

In [16]:
import pandas as pd

# Assuming df is your DataFrame and "AN_PROC_NAME" is your column
df['word_count'] = df['AN_PROC_NAME'].apply(lambda x: len(str(x).split()))
average_word_count = df['word_count'].mean()

print("Average word count:", average_word_count)

Average word count: 8.899134020618556


In [17]:
# Assuming df is your DataFrame and "AN_PROC_NAME" is your column
unique_words = set()

df['AN_PROC_NAME'].apply(lambda x: unique_words.update(str(x).split()))

print("Number of unique words:", len(unique_words))

Number of unique words: 3203


In [18]:
len(df)

84875

In [4]:
def K_fold_val(embedding_name, outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # Define the parameter grid
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]}
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df,df["PE"])):
        # Split the data into train/test sets
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"{embedding_name}-fold{i}.pickle"
        i=i+1
        if os.path.exists(name):
            with open(name, "rb") as file:
                d = pickle.load(file)
            X_train=d[0]
            X_test=d[1]
        else:
            # this extracts the embeddings for us via the embeddings.py read earlier
            X_train = identify_embeddings(embedding_name, train, 'AN_PROC_NAME')
            X_test = identify_embeddings(embedding_name, test, 'AN_PROC_NAME')
            with open(name, 'wb') as f:
                pickle.dump([X_train,X_test], f)
        
        # Prepare training and  testing data
        y_train = train[outcome_col]
        y_test = test[outcome_col]
        
        # drop potential NAs. 
        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        # perform XGBoost classification
        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        # #print(grid_search.best_params_)
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)

    with open(f"{embedding_name}_{outcome_col}.pickle", 'wb') as f:
        pickle.dump([auroc_scores,auprc_scores], f)
    

    # Now you can calculate the mean and standard deviation
    mean_auroc=np.mean(auroc_scores)
    std_auroc=np.std(auroc_scores)
    mean_auprc=np.mean(auprc_scores)
    std_auprc=np.std(auprc_scores)

    
    return(f"{embedding_name} metrics: Mean AUROC: {mean_auroc}, std: {std_auroc}, Mean AUPRC: {mean_auprc}, std: {std_auprc}")


## Part I: Death in 30

In [7]:
list_of_results, embeddings=[],["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"]
#list_of_results, embeddings=list_of_results,["glove"]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, "death_in_30", df)
    # print(results)
    list_of_results.append(results)
    with open("death_30_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)

In [ ]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.8617310089212685, SD AUROC: 0.007396454689913205, Mean AUPRC: 0.16144970531277464, SD AUPRC: 0.014679291974249404', 'clinicalBERT metrics: Mean AUROC: 0.8546206099816303, SD AUROC: 0.008751427733780633, Mean AUPRC: 0.1548581910521805, SD AUPRC: 0.012955381081773949', 'bioclinicalBERT metrics: Mean AUROC: 0.8502950231779259, SD AUROC: 0.007691225365106731, Mean AUPRC: 0.1558769766743032, SD AUPRC: 0.01264557217842545', 'cbow metrics: Mean AUROC: 0.5283590226217163, SD AUROC: 0.08604701468519638, Mean AUPRC: 0.02268354022936795, SD AUPRC: 0.005876665012416533', 'doc2vec metrics: Mean AUROC: 0.47916207658847937, SD AUROC: 0.09479093360430248, Mean AUPRC: 0.02077291969133726, SD AUPRC: 0.006394345460844289', 'fasttext metrics: Mean AUROC: 0.7252647655436114, SD AUROC: 0.03988160961203121, Mean AUPRC: 0.049825086838406994, SD AUPRC: 0.007337991905855', 'glove metrics: Mean AUROC: 0.8180773201935911, SD AUROC: 0.008295612437432268, Mean AUPRC: 0.12825155959231

## part II: DVT

In [ ]:
import pickle
embeddings,list_of_results=["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"],[]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, "DVT", df)
    # print(results)
    list_of_results.append(results)
    with open("DVT_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)

In [ ]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.773348145994715, SD AUROC: 0.028273693639845467, Mean AUPRC: 0.024151678287604124, SD AUPRC: 0.005899939528372898', 'clinicalBERT metrics: Mean AUROC: 0.7644036506167788, SD AUROC: 0.024910883107822513, Mean AUPRC: 0.022450969674043402, SD AUPRC: 0.005162268680841622', 'bioclinicalBERT metrics: Mean AUROC: 0.7597950514122236, SD AUROC: 0.02641536673069206, Mean AUPRC: 0.020410492455534324, SD AUPRC: 0.004597984973915123', 'cbow metrics: Mean AUROC: 0.5236231949676923, SD AUROC: 0.04815035348339372, Mean AUPRC: 0.006118309745923913, SD AUPRC: 0.0007153847011203043', 'doc2vec metrics: Mean AUROC: 0.5306616126257037, SD AUROC: 0.0633018634433623, Mean AUPRC: 0.00707150602233197, SD AUPRC: 0.0023001339850773827', 'fasttext metrics: Mean AUROC: 0.6938415900626116, SD AUROC: 0.03762486541425244, Mean AUPRC: 0.014013294857429809, SD AUPRC: 0.0023164483235037624', 'glove metrics: Mean AUROC: 0.7228278270751121, SD AUROC: 0.016268028748881586, Mean AUPRC: 0.01870

## Part III: PE

In [ ]:
list_of_results, embeddings=[],["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, "PE", df)
    # print(results)
    list_of_results.append(results)
    with open("PE_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)

In [ ]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.7108299880980891, SD AUROC: 0.022940564002046904, Mean AUPRC: 0.010913892319031837, SD AUPRC: 0.004108470927532611', 'clinicalBERT metrics: Mean AUROC: 0.7171382372798917, SD AUROC: 0.018939027237084148, Mean AUPRC: 0.012981865424277927, SD AUPRC: 0.002696057681125479', 'bioclinicalBERT metrics: Mean AUROC: 0.682770573510368, SD AUROC: 0.044557726230357315, Mean AUPRC: 0.008397340419296842, SD AUPRC: 0.0015269635350937142', 'cbow metrics: Mean AUROC: 0.505670090555504, SD AUROC: 0.06314640446138049, Mean AUPRC: 0.0038775163857217936, SD AUPRC: 0.0012303162595203854', 'doc2vec metrics: Mean AUROC: 0.5166633823767992, SD AUROC: 0.0362113185949415, Mean AUPRC: 0.0038067599963316516, SD AUPRC: 0.00014964834806831327', 'fasttext metrics: Mean AUROC: 0.651630331050766, SD AUROC: 0.03559425425035086, Mean AUPRC: 0.007475830368225568, SD AUPRC: 0.0016313530431375323', 'glove metrics: Mean AUROC: 0.6643808878552064, SD AUROC: 0.026201407470637643, Mean AUPRC: 0.0

## PNA

In [ ]:
list_of_results, embeddings=[],["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, 'PNA', df)
    # print(results)
    list_of_results.append(results)
    with open("PNA_outcome_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)
print(list_of_results)

In [ ]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.8182941374496314, SD AUROC: 0.013462397429968073, Mean AUPRC: 0.047320444126914676, SD AUPRC: 0.007682162389743574', 'clinicalBERT metrics: Mean AUROC: 0.8056188002552421, SD AUROC: 0.015262786080245219, Mean AUPRC: 0.03999518406633077, SD AUPRC: 0.011779019719213931', 'bioclinicalBERT metrics: Mean AUROC: 0.8092945477477101, SD AUROC: 0.017273743100448012, Mean AUPRC: 0.042933651015547594, SD AUPRC: 0.01126089572612011', 'cbow metrics: Mean AUROC: 0.5255906784421045, SD AUROC: 0.10228195093733464, Mean AUPRC: 0.008688124960603697, SD AUPRC: 0.0052415725418974955', 'doc2vec metrics: Mean AUROC: 0.4948302021650548, SD AUROC: 0.10652549739930975, Mean AUPRC: 0.00649413124674682, SD AUPRC: 0.0027075315163265783', 'fasttext metrics: Mean AUROC: 0.6959422333246963, SD AUROC: 0.038061431750688074, Mean AUPRC: 0.015564539291985762, SD AUPRC: 0.005764301684785904', 'glove metrics: Mean AUROC: 0.7653026083580113, SD AUROC: 0.024186438303324873, Mean AUPRC: 0.0404

## AKI

In [5]:
list_of_results, embeddings=[],["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, 'post_aki_status', df)
    # print(results)
    list_of_results.append(results)
    with open("post_aki_status_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)

  0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   2.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.9s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   2.5s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.5s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.1s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [12:57, 155.58s/it]
100%|██████████| 7/7 [1:49:19<00:00, 937.05s/it]


In [6]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.8351797726689462, std: 0.0018533738644413422, Mean AUPRC: 0.4783178062818684, std: 0.009880074970242556', 'clinicalBERT metrics: Mean AUROC: 0.8297936798022004, std: 0.0021013822642247377, Mean AUPRC: 0.4691577910581004, std: 0.007787899086533284', 'bioclinicalBERT metrics: Mean AUROC: 0.8295033678747391, std: 0.001279832487445412, Mean AUPRC: 0.46881892900678734, std: 0.008175478983831952', 'cbow metrics: Mean AUROC: 0.5603038304987245, std: 0.05191323417599398, Mean AUPRC: 0.1560744283944533, std: 0.022292049088444406', 'doc2vec metrics: Mean AUROC: 0.5226496762137398, std: 0.07299570105465811, Mean AUPRC: 0.145904164106554, std: 0.03849242086794131', 'fasttext metrics: Mean AUROC: 0.7260719660046424, std: 0.017507946185940454, Mean AUPRC: 0.27296768226350643, std: 0.02464008813443889', 'glove metrics: Mean AUROC: 0.8097660210920864, std: 0.0035123924002372297, Mean AUPRC: 0.4406563640978164, std: 0.0075252290049158755']


## Delirium

In [7]:
list_of_results, embeddings=[],["bioGPT","clinicalBERT","bioclinicalBERT","cbow","doc2vec","fasttext","glove"]
for embedding_name in tqdm(embeddings):
    results=K_fold_val(embedding_name, 'postop_del', df)
    # print(results)
    list_of_results.append(results)
    with open("postop_del_update.pickle", 'wb') as f:
        pickle.dump(list_of_results, f)

  0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.4s
[CV] END .learning_rate=0.1, max_depth=4, min_c

...

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.2s
[CV] END .learning_rate=0.1, max_depth=4, min_c

5it [07:28, 89.70s/it]
100%|██████████| 7/7 [57:08<00:00, 489.79s/it]


In [8]:
print(list_of_results)

['bioGPT metrics: Mean AUROC: 0.6909620237431988, std: 0.013508536467064703, Mean AUPRC: 0.6636434420901307, std: 0.018691321440030953', 'clinicalBERT metrics: Mean AUROC: 0.6863633334545323, std: 0.011288866708364783, Mean AUPRC: 0.6600336651807283, std: 0.018984719599622907', 'bioclinicalBERT metrics: Mean AUROC: 0.6801388815960356, std: 0.01206336415983351, Mean AUPRC: 0.6528951711744715, std: 0.019446049543426724', 'cbow metrics: Mean AUROC: 0.5011428045750416, std: 0.026992335730944637, Mean AUPRC: 0.4736543974480143, std: 0.02380529759246651', 'doc2vec metrics: Mean AUROC: 0.48449546038789926, std: 0.032530971340643575, Mean AUPRC: 0.46575595393554536, std: 0.03497823650477376', 'fasttext metrics: Mean AUROC: 0.564633647598688, std: 0.017219893590980126, Mean AUPRC: 0.5331354423256298, std: 0.014863764737777852', 'glove metrics: Mean AUROC: 0.6664023021714881, std: 0.010494102314867434, Mean AUPRC: 0.6363763643979465, std: 0.01674996512283214']
